In [288]:
import pandas as pd
import numpy as np
from IPython.display import display
from fuzzywuzzy import fuzz
import faker
import uuid

# 2 Way Match Demo

## Generate Fake PO Data

In [289]:
# generate fake purchase order data using faker
fake = faker.Faker(seed=4331)

sample_material = [
    "electrical box and enclosure and fitting and accessory",
    "electrical lug plug and connector and accessory",
    "electrical plug",
    "electrical receptacle",
    "electrical control and accessory",
    "push button switch",
    "circuit protection device and accessory",
    "electrical hardware and supply",
    "electrical hanger",
    "manufacture component and supply",
    "casting and cast assembly",
]

po = {
    "po_id": "",
    "po_date": "",
    "po_vendor": "",
    "items": {
        "item_id": "",
        "item_amount": 0,
        "item_currency": "",
        "item_description": "",
        "item_quantity": 0,
    },
}


def generate_po_data():
    all_po_data = []
    for i in range(5):
        new_po = po.copy()
        new_po["po_id"] = uuid.uuid4().hex[0:8].upper()
        new_po["po_date"] = fake.date_between(start_date="-1y", end_date="today")
        new_po["po_vendor"] = fake.company()
        po_currency = fake.currency_code()

        # generate random number of po lines
        total_po_lines = fake.pyint(min_value=1, max_value=10, step=1)

        items = []

        for j in range(total_po_lines):
            # get all item descriptions from items list
            desc_used = [x["item_description"] for x in items]


            items.append(
                {
                    "item_id": j + 1,
                    "item_amount": fake.pyfloat(
                        left_digits=5, right_digits=2, positive=True
                    ),
                    "item_currency": po_currency,
                    "item_quantity": fake.pyint(min_value=1, max_value=100, step=1),

                    # make sure the item description is unique
                    "item_description": fake.random_element([x for x in sample_material if x not in desc_used]),
                }
            )

        new_po["items"] = items
        all_po_data.append(new_po)

    return all_po_data


# create a dataframe from dictionary by flattening the items column
df = pd.json_normalize(
    generate_po_data(), record_path="items", meta=["po_id", "po_date", "po_vendor"]
)

# reordering the columns to show po data first
df = df[
    [
        "po_id",
        "po_date",
        "po_vendor",
        "item_id",
        "item_description",
        "item_amount",
        "item_currency",
        "item_quantity",
    ]
]

df


,po_id,po_date,po_vendor,item_id,item_description,item_amount,item_currency,item_quantity
0,C2D71FFB,2022-05-26,Williams Ltd,1,electrical lug plug and connector and accessory,79135.13,HNL,78
1,C2D71FFB,2022-05-26,Williams Ltd,2,circuit protection device and accessory,98018.95,HNL,76
2,C2D71FFB,2022-05-26,Williams Ltd,3,push button switch,44296.67,HNL,36
3,C2D71FFB,2022-05-26,Williams Ltd,4,electrical receptacle,40980.36,HNL,94
4,C2D71FFB,2022-05-26,Williams Ltd,5,electrical control and accessory,5376.74,HNL,41
5,C2D71FFB,2022-05-26,Williams Ltd,6,electrical box and enclosure and fitting and a...,38827.35,HNL,76
6,C2D71FFB,2022-05-26,Williams Ltd,7,electrical hanger,3985.42,HNL,10
7,3D39749A,2022-12-06,Lin-Murray,1,electrical hanger,26283.82,RSD,16
8,3D39749A,2022-12-06,Lin-Murray,2,electrical control and accessory,74006.97,RSD,34
9,3D39749A,2022-12-06,Lin-Murray,3,electrical lug plug and connector and accessory,98343.58,RSD,16


Some Stats on PO Data

In [290]:
# show count of items per po
df.groupby('po_id')['item_id'].count().reset_index(name='item_count')

,po_id,item_count
0,1BBC022A,7
1,3D39749A,6
2,C2D71FFB,7
3,F798E45B,7
4,F8AB7C8B,6


## Generate Invoice data by randomizing PO Data

In [291]:
invoice = {
    "invoice_id": "",
    "po_id": "",
    "invoice_date": "",
    "invoice_vendor": "",
    "items": {
        "item_id": "",
        "item_amount": 0,
        "item_currency": "",
        "item_description": "",
        "item_quantity": 0,
    },
}

# Generate invoice data by altering some of the PO data
def generate_invoice_data(po_df):
    all_invoice_data = []
    # generate new invoice for each po_id
    for po_id in po_df['po_id'].unique():
        # get po data for each po_id
        po_data = po_df[po_df['po_id'] == po_id]
        # create a new invoice
        new_invoice = invoice.copy()
        new_invoice['invoice_id'] = uuid.uuid4().hex[0:8].upper()
        new_invoice['po_id'] = po_id
        # generate a random date between the po date and today
        new_invoice['invoice_date'] = fake.date_between(start_date=po_data['po_date'].unique()[0], end_date='today')
        new_invoice['invoice_vendor'] = po_data['po_vendor'].unique()[0]
        # create a new list to hold the items
        items = []
        # generate new item for invoice that can be more or less than the po
        total_invoice_lines = fake.pyint(min_value=1, max_value=10, step=1)
        for j in range(total_invoice_lines):
            items.append(
                {
                    "item_id": j + 1,
                    # generate a boolean to determine if the item amount should be the same as the po
                    "item_amount": po_data['item_amount'].unique()[0] if fake.pybool() else fake.pyfloat(left_digits=5, right_digits=2, positive=True),
                    # generate a boolean to determine if the item currency should be the same as the po
                    "item_currency": po_data['item_currency'].unique()[0] if fake.pybool() else fake.currency_code(),
                    # generate a boolean to determine if the item quantity should be the same as the po or less
                    "item_quantity": po_data['item_quantity'].unique()[0] if fake.pybool() else fake.pyint(min_value=1, max_value=po_data['item_quantity'].unique()[0], step=1),
                    # generate a boolean to determine if the item description should be the same as the po but dont repeat the same description
                    "item_description": po_data['item_description'].unique()[0] if fake.pybool() else fake.random_element([x for x in sample_material if x not in po_data['item_description'].unique()]),
                }
            )

        new_invoice['items'] = items
        all_invoice_data.append(new_invoice)
    return all_invoice_data

# create a dataframe from dictionary by flattening the items column
invoice_df = pd.json_normalize(generate_invoice_data(df), record_path='items', meta=['invoice_id', "po_id", 'invoice_date', 'invoice_vendor'])

# reordering the columns to show invoice data first
invoice_df = invoice_df[['invoice_id', "po_id", 'invoice_date', 'invoice_vendor', 'item_id', 'item_description', 'item_amount', 'item_currency', 'item_quantity']]

invoice_df


,invoice_id,po_id,invoice_date,invoice_vendor,item_id,item_description,item_amount,item_currency,item_quantity
0,236D9F0A,C2D71FFB,2022-08-20,Williams Ltd,1,electrical plug,97343.20,NPR,69
1,236D9F0A,C2D71FFB,2022-08-20,Williams Ltd,2,electrical hardware and supply,92121.86,UAH,78
2,1296A6E0,3D39749A,2023-01-16,Lin-Murray,1,manufacture component and supply,26283.82,USD,16
3,1296A6E0,3D39749A,2023-01-16,Lin-Murray,2,electrical hanger,26283.82,RSD,15
4,4E7F460D,F798E45B,2022-11-24,"Taylor, Barnett and Stewart",1,electrical lug plug and connector and accessory,4718.57,KMF,7
5,4E7F460D,F798E45B,2022-11-24,"Taylor, Barnett and Stewart",2,electrical hardware and supply,24995.50,KMF,24
6,4E7F460D,F798E45B,2022-11-24,"Taylor, Barnett and Stewart",3,push button switch,82231.98,NAD,27
7,4E7F460D,F798E45B,2022-11-24,"Taylor, Barnett and Stewart",4,push button switch,14271.48,CDF,21
8,4E7F460D,F798E45B,2022-11-24,"Taylor, Barnett and Stewart",5,electrical hardware and supply,14271.48,KMF,32
9,4E7F460D,F798E45B,2022-11-24,"Taylor, Barnett and Stewart",6,casting and cast assembly,14271.48,MUR,32


## Go through each PO to validate the matching line items

In [292]:
# loop through each invoice and compare the items to the po
for invoice_id in invoice_df['invoice_id'].unique():
    # get invoice data for each invoice_id
    invoice_data = invoice_df[invoice_df['invoice_id'] == invoice_id]
    # get po data for each invoice_id
    po_data = df[df['po_id'] == invoice_data['po_id'].unique()[0]]
    # Compare if the total items in the invoice is the same as the po and provide total items in invoice and po
    if invoice_data['item_id'].count() == po_data['item_id'].count():
        print(f'Invoice {invoice_id} has the same number of items as the PO: {invoice_data["item_id"].count()} items')
    else:
        print(f'Invoice {invoice_id} has a different number of items than the PO. Invoice has {invoice_data["item_id"].count()} items and PO has {po_data["item_id"].count()} items')

Invoice 236D9F0A has a different number of items than the PO. Invoice has 2 items and PO has 7 items
Invoice 1296A6E0 has a different number of items than the PO. Invoice has 2 items and PO has 6 items
Invoice 4E7F460D has a different number of items than the PO. Invoice has 8 items and PO has 7 items
Invoice 86DC4D84 has a different number of items than the PO. Invoice has 8 items and PO has 7 items
Invoice 74DB396D has a different number of items than the PO. Invoice has 7 items and PO has 6 items


## Compare Invoice Line Items with PO Line Items

In [296]:
# Loop through each invoice and compare the items to the po
for invoice_id in invoice_df['invoice_id'].unique():
    # get invoice data for each invoice_id
    invoice_data = invoice_df[invoice_df['invoice_id'] == invoice_id]
    
    # get po data for each invoice_id
    po_data = df[df['po_id'] == invoice_data['po_id'].unique()[0]]

    # get only item data from invoice
    invoice_item_data = invoice_data[['item_id', 'item_description', 'item_amount', 'item_quantity']]

    # get only item data from po
    po_item_data = po_data[['item_id', 'item_description', 'item_amount', 'item_quantity']]

    # create a cross join of the invoice and po items
    invoice_po_item_data = pd.merge(invoice_item_data, po_item_data, how='cross', suffixes=('_invoice', '_po'))

    # Calculate percentage difference between invoice and po item amounts with invoice amount as the base rounded to 2 decimal places
    invoice_po_item_data['amount_diff'] = round((invoice_po_item_data['item_amount_po'] - invoice_po_item_data['item_amount_invoice']) / invoice_po_item_data['item_amount_invoice'] * 100, 2)

    # Calculate percentage difference between invoice and po item quantities with invoice quantity as the base rounded to 2 decimal places
    invoice_po_item_data['quantity_diff'] = round((invoice_po_item_data['item_quantity_po'] - invoice_po_item_data['item_quantity_invoice']) / invoice_po_item_data['item_quantity_invoice'] * 100, 2)

    # Calculate text difference between invoice and po item descriptions using fuzzywuzzy
    invoice_po_item_data['description_diff'] = invoice_po_item_data.apply(lambda x: 100 - fuzz.ratio(x['item_description_invoice'], x['item_description_po']), axis=1)

    # calculate the weighted average of the absolute values of amount, quantity, and description differences
    invoice_po_item_data['weighted_diff'] = round(abs((invoice_po_item_data['amount_diff']) * 0.25) + abs(invoice_po_item_data['quantity_diff'] * 0.25) + abs(invoice_po_item_data['description_diff'] * 0.50), 2)

    paired_items = []

    # sort the invoice_po_item_data by weighted_diff
    invoice_po_item_data = invoice_po_item_data.sort_values(by='weighted_diff', ascending=True)

    # loop through each row in invoice_po_item_data
    for index, row in invoice_po_item_data.iterrows():
        # get the item_id_invoice
        item_id_invoice = row['item_id_invoice']

        # get the po_item_id
        po_item_id = row['item_id_po']

        # get weighted_diff for the row
        weighted_diff = row['weighted_diff']

        # if check both the item_id_invoice and po_item_id are not in paired_items
        if (item_id_invoice not in [x[0] for x in paired_items]) and (po_item_id not in [x[1] for x in paired_items]):
            # check if weighted_diff is less than 50
            # if weighted_diff < 50:
             # if yes, add the item_id_invoice and po_item_id to paired_items
            paired_items.append((item_id_invoice, po_item_id, weighted_diff))
        else:
            # if present, skip the row
            continue
   
    # select new colum in invoice_po_item_data to determine if the item is paired
    invoice_po_item_data['paired'] = invoice_po_item_data.apply(lambda x: True if (x['item_id_invoice'], x['item_id_po'], x['weighted_diff']) in paired_items else False, axis=1)

    # save the invoice_po_item_data to a csv file
    invoice_po_item_data.to_csv(f'invoice_{invoice_id}.csv', index=False)

    # print the number of paired items
    print("-"*50)
    # print total invoice items
    print(f'Invoice {invoice_id} has {invoice_data["item_id"].count()} items')
    # print total po items
    po_id = invoice_data['po_id'].unique()[0]
    # print total po items from original po data
    print(f'PO {po_id} has {df[df["po_id"] == po_id]["item_id"].count()} items')
    print(f'Invoice {invoice_id} has {len(paired_items)} paired items')
    print("-"*50)
    
    # display the matched items
    display(invoice_po_item_data[invoice_po_item_data['paired'] == True].sort_values(by='weighted_diff', ascending=True))

--------------------------------------------------
Invoice 236D9F0A has 2 items
PO C2D71FFB has 7 items
Invoice 236D9F0A has 2 paired items
--------------------------------------------------


,item_id_invoice,item_description_invoice,item_amount_invoice,item_quantity_invoice,item_id_po,item_description_po,item_amount_po,item_quantity_po,amount_diff,quantity_diff,description_diff,weighted_diff,paired
7,2,electrical hardware and supply,92121.86,78,1,electrical lug plug and connector and accessory,79135.13,78,-14.1,0.00,45,26.02,True
3,1,electrical plug,97343.20,69,4,electrical receptacle,40980.36,94,-57.9,36.23,28,37.53,True


--------------------------------------------------
Invoice 1296A6E0 has 2 items
PO 3D39749A has 6 items
Invoice 1296A6E0 has 2 paired items
--------------------------------------------------


,item_id_invoice,item_description_invoice,item_amount_invoice,item_quantity_invoice,item_id_po,item_description_po,item_amount_po,item_quantity_po,amount_diff,quantity_diff,description_diff,weighted_diff,paired
6,2,electrical hanger,26283.82,15,1,electrical hanger,26283.82,16,0.0,6.67,0,1.67,True
3,1,manufacture component and supply,26283.82,16,4,electrical box and enclosure and fitting and a...,50017.97,5,90.3,-68.75,63,71.26,True


--------------------------------------------------
Invoice 4E7F460D has 8 items
PO F798E45B has 7 items
Invoice 4E7F460D has 7 paired items
--------------------------------------------------


,item_id_invoice,item_description_invoice,item_amount_invoice,item_quantity_invoice,item_id_po,item_description_po,item_amount_po,item_quantity_po,amount_diff,quantity_diff,description_diff,weighted_diff,paired
21,4,push button switch,14271.48,21,1,push button switch,14271.48,32,0.00,52.38,0,13.10,True
11,2,electrical hardware and supply,24995.50,24,5,electrical hanger,27457.22,31,9.85,29.17,40,29.76,True
34,5,electrical hardware and supply,14271.48,32,7,electrical control and accessory,10553.64,53,-26.05,65.62,39,42.42,True
54,8,electrical hardware and supply,94890.59,26,6,manufacture component and supply,5731.93,24,-93.96,-7.69,48,49.41,True
16,3,push button switch,82231.98,27,3,circuit protection device and accessory,33452.78,17,-59.32,-37.04,68,58.09,True
45,7,electrical receptacle,14271.48,26,4,electrical plug,58739.19,50,311.58,92.31,28,114.97,True
36,6,casting and cast assembly,14271.48,32,2,electrical box and enclosure and fitting and a...,89730.80,85,528.74,165.62,59,203.09,True


--------------------------------------------------
Invoice 86DC4D84 has 8 items
PO 1BBC022A has 7 items
Invoice 86DC4D84 has 7 paired items
--------------------------------------------------


,item_id_invoice,item_description_invoice,item_amount_invoice,item_quantity_invoice,item_id_po,item_description_po,item_amount_po,item_quantity_po,amount_diff,quantity_diff,description_diff,weighted_diff,paired
21,4,electrical hanger,83812.50,52,1,electrical hanger,83812.50,52,0.00,0.00,0,0.00,True
51,8,electrical lug plug and connector and accessory,77613.59,26,3,circuit protection device and accessory,99942.79,16,28.77,-38.46,42,37.81,True
3,1,electrical box and enclosure and fitting and a...,83812.50,52,4,electrical control and accessory,86085.96,98,2.71,88.46,33,39.29,True
20,3,manufacture component and supply,83812.50,20,7,electrical hardware and supply,58154.60,13,-30.61,-35.00,48,40.40,True
32,5,manufacture component and supply,83812.50,52,5,casting and cast assembly,56983.11,71,-32.01,36.54,61,47.64,True
8,2,electrical hanger,83812.50,27,2,push button switch,5269.61,17,-93.71,-37.04,77,71.19,True
40,6,electrical box and enclosure and fitting and a...,27102.77,52,6,electrical receptacle,88544.52,99,226.70,90.38,52,105.27,True


--------------------------------------------------
Invoice 74DB396D has 7 items
PO F8AB7C8B has 6 items
Invoice 74DB396D has 6 paired items
--------------------------------------------------


,item_id_invoice,item_description_invoice,item_amount_invoice,item_quantity_invoice,item_id_po,item_description_po,item_amount_po,item_quantity_po,amount_diff,quantity_diff,description_diff,weighted_diff,paired
36,7,casting and cast assembly,51875.59,7,1,casting and cast assembly,51875.59,7,0.00,0.00,0,0.00,True
26,5,electrical receptacle,89339.46,4,3,electrical plug,42015.71,2,-52.97,-50.00,28,39.74,True
1,1,casting and cast assembly,76147.97,7,2,electrical lug plug and connector and accessory,86517.44,24,13.62,242.86,56,92.12,True
23,4,casting and cast assembly,42966.80,7,6,electrical hardware and supply,36658.15,44,-14.68,528.57,56,163.81,True
34,6,casting and cast assembly,51875.59,7,5,circuit protection device and accessory,94391.68,59,81.96,742.86,59,235.70,True
9,2,electrical box and enclosure and fitting and a...,14642.14,7,4,push button switch,25223.32,80,72.27,1042.86,75,316.28,True
